## **Notebook 04: Gradio Web Interface**

### Purpose
Build a polished, user-facing web interface for Radar using Gradio. This transforms our agent pipeline into an accessible demo that can be deployed and shared.

### What We'll Do

| Step | Task | Output |
|------|------|--------|
| 1 | **Setup Gradio** | Import libraries and configure interface |
| 2 | **Design Layout** | Create card-based UI with sections |
| 3 | **Build Processing Function** | Connect Gradio to agent pipeline |
| 4 | **Add ArXiv Input** | Allow users to input paper IDs |
| 5 | **Style Interface** | Custom CSS for professional appearance |
| 6 | **Test Locally** | Run and validate interface |
| 7 | **Deployment Prep** | Package for Hugging Face Spaces |

### Key Questions to Answer
- How do we structure the output for maximum readability?
- What input methods work best (ArXiv ID vs file upload)?
- How do we handle loading states and errors gracefully?
- What styling makes this look portfolio-quality?

### Expected Outcomes
- Working Gradio interface running locally
- Clean, card-based layout for results
- Professional styling with gradients and spacing
- Ready-to-deploy application file

### Design Philosophy
Focus on clarity and scannability. Users should immediately understand:
1. What the paper is about (Two-line summary)
2. What problem it solves (Challenge)
3. How it solves it (Solution)
4. Technical details (Key points)

---



In [4]:
# Cell 2: Imports and Setup

"""
Import Gradio and connect to our existing agent pipeline.
"""

import gradio as gr
import json
import os
from datetime import datetime

# LangGraph and agents
from langgraph.graph import StateGraph, END
from langchain_anthropic import ChatAnthropic
from typing import TypedDict

# ArXiv
import arxiv

# PDF processing
import fitz

# Environment
from dotenv import load_dotenv
load_dotenv()

# Verify API key
api_key = os.getenv('ANTHROPIC_API_KEY')
if not api_key:
    raise ValueError("ANTHROPIC_API_KEY not found")

# Initialize Claude client
llm = ChatAnthropic(
    model="claude-sonnet-4-20250514",
    temperature=0.7,
    max_tokens=4096
)

print("Setup complete")
print(f"Gradio version: {gr.__version__}")
print("Ready to build interface")

Setup complete
Gradio version: 6.3.0
Ready to build interface
